# TfL Data Downloading

__December 2022__

This notebook is to test out downloading data directly from the TfL website, will move to a .py file.

In [100]:
import re
import os
import yaml
import requests
import pandas as pd

from io import StringIO
from yaml import Loader

In [101]:
params = yaml.load(open("../params.yaml", 'r'), Loader=Loader)

In [61]:
# with requests.Session() as session: 
#     for link in params['data_links']:
#         print(link)
#         download = session.get(link)
        
#         df = pd.read_csv(
#             StringIO(download.content.decode(encoding='utf-8', errors='replace')),
#             encoding= 'unicode_escape',
#             low_memory=False
#         )

#         if 'vehicle' in link:
#             file_type = 'vehicles'
#         elif 'casualty' in link:
#             file_type = 'casualties'
#         elif 'attendant' in link:
#             file_type = 'collisions'
            
#         display(df.head())
#         print(df.shape)

#         year = re.findall("20\d{2}", link)[0]
        
#         df.to_csv(f'../data/raw-data/{file_type}-{year}.csv', index=False)

In [5]:
# test random
pd.read_csv('../data/raw-data/collisions-2014.csv').head()

,AREFNO,Borough,Boro,Easting,Northing,Location,Accident Severity,No. of Casualties in Acc.,No. of Vehicles in Acc.,Accident Date,...,Junction Detail,Junction Control,Road Class 2,Road No. 2,Ped. Crossing Decoded,Light Conditions (Banded),Weather,Road Surface,Special Conditions,C/W Hazard
0,0114CP00001,CITY OF LONDON,0,533540,181230,ST BOTOLPH STREET J/W HOUNDSDITCH,3 Slight,1,1,01-Jan-14,...,3 T/Stag Jun,4 Give Way/Uncontrolled,A,1211,0 No Xing Facility In 50m,2 Dark,1 Fine,2 Road-Wet,0 None,0 None
1,0114CP00002,CITY OF LONDON,0,532680,181430,MOORGATE J/W GREAT SWAN ALLEY,3 Slight,1,1,08-Jan-14,...,3 T/Stag Jun,4 Give Way/Uncontrolled,C,0,0 No Xing Facility In 50m,1 Daylight,1 Fine,1 Road-Dry,0 None,0 None
2,0114CP00003,CITY OF LONDON,0,532090,181830,ALDERGATE STREET J/W LONG LANE,2 Serious,1,1,09-Jan-14,...,3 T/Stag Jun,2 Auto Sig,C,0,5 Pedn Phase At Ats,2 Dark,1 Fine,1 Road-Dry,0 None,0 None
3,0114CP00004,CITY OF LONDON,0,531770,180950,QUEEN VICTORIA STREET J/W PUDDLE DOCK,2 Serious,1,2,08-Jan-14,...,3 T/Stag Jun,2 Auto Sig,C,0,5 Pedn Phase At Ats,1 Daylight,1 Fine,1 Road-Dry,0 None,0 None
4,0114CP00005,CITY OF LONDON,0,533130,180920,FENCHURCH STREET J/W ROOD LANE,3 Slight,1,1,15-Jan-14,...,3 T/Stag Jun,4 Give Way/Uncontrolled,C,0,0 No Xing Facility In 50m,1 Daylight,1 Fine,1 Road-Dry,0 None,0 None


In [102]:
def extract_columns(df, schema):
    inverse_schema = {}
    for key, val in schema.items():
        for v in val:
            inverse_schema[v] = key
        
    df = df.rename(columns=inverse_schema)
    columns = schema.keys()
    return df[columns]


def format_name(name):
    name = name.strip() # strip trailing spaces
    name = name.lower()
    name = name.lstrip('_')  # string underscores infront of col name
    name = re.sub('/', '_or_', name)
    name = re.sub(r'[^\w\s]', '', name)  # remove punctuation
    name = name.replace(' ', '_')  # replace spaces between words w/ underscores
    
    return name


def format_category(val, categories):
    val = format_name(val)
    for category in categories:
        if category in val:
            return category
    # return column if no matches found
    print(f'No matches found for: {val}')
    return val
    
    
def format_time(time):
    """
    Format time if in format '0731 (rather than 07:31)
    """
    if time[0] == "'":
        time = f'{time[1:3]}:{time[3:]}'
    
    return time


def process_yearly_data(path, data_type, schema):
    files = [f for f in os.listdir(path) if data_type in f]
    
    dfs = []
    for file in files:
        n = 0
        cols = ['Unnamed:']
        while len([c for c in cols if 'Unnamed:' in c]) > 0:
            df = pd.read_csv(f'{path}/{file}', skiprows=n)
            cols = df.columns
            n += 1

        df.columns = [col.strip() for col in df.columns]

        df = extract_columns(df, schema)

        dfs.append(df)

    combined_df = pd.concat(dfs)
    return combined_df

In [103]:
severities = [
    'slight',
    'serious',
    'fatal'
]

casualty_classes = [
    'driver_or_rider',
    'passenger',
    'pedestrian'
]

modes_of_travel = [
    "pedestrian",
    "pedal_cycle",
    "powered_2_wheeler",
    "car",
    "taxi",
    "bus_or_coach",
    "goods_vehicle",
    "other_vehicle",
    "private_hire"
]

In [104]:
collision_schema = params['collision_columns']
casualty_schema = params['casualty_columns']

In [109]:
collisions = process_yearly_data(
    path='../data/raw-data/',
    data_type='collisions',
    schema=collision_schema
)

collisions['date'] = pd.to_datetime(collisions['date'], infer_datetime_format=True, dayfirst=True)

collisions['time'] = collisions['time'].apply(format_time)
collisions['time'] = pd.to_datetime(collisions['time']).dt.time

# collisions['collision_severity'] = collisions['collision_severity'].apply(
#     lambda x: format_category(x, severities)
# )

for col in ['borough', 'location']:
    collisions[col] = collisions[col].apply(lambda x: x.upper())

collisions.sample(5)

,collision_id,borough,easting,northing,location,collision_severity,junction_detail,date,time
12469,0113KG70077,BARKING & DAGENHAM,549040,184850,HEATHWAY J/W PARSOLES AVE,2 Serious,6 Crossroads,2013-02-14,16:04:00
24647,0115YE80331,ENFIELD,534380,197490,GREAT CAMBRIDGE RD J/W CARTERHATCH LANE,3 Slight,6 Crossroads,2015-04-07,18:55:00
23805,0112YE80583,ENFIELD,529620,194310,NFL CHASE SIDE 25 M W J/W STATION PARADE,3 Slight,0 No Jun In 20m,2012-09-10,18:15:00
22064,0111SX20367,BARNET,524390,186820,HENDON WAY J/W THE VALE,3 Slight,6 Crossroads,2011-05-30,17:50:00
19139,1210341857,HACKNEY,533029,186172,"ON ALBION ROAD, 100 METRES NORTH OF THE JUNCTI...",Slight,No Jun In 20m,2021-11-05,10:35:00


In [110]:
list(collisions.collision_severity.unique())

['3 Slight',
 '2 Serious',
 '1 Fatal',
 '3 SLIGHT',
 '2 SERIOUS',
 '1 FATAL',
 'Slight',
 'Serious',
 'Fatal']

In [108]:
list(collisions.junction_detail.unique())

['3 T/Stag Jun',
 '6 Crossroads',
 '9 Other Jun',
 '7 Multi Jun',
 '0 No Jun In 20m',
 '1 Roundabout',
 '5 Slip Road',
 '8 Priv Drive',
 '2 Mini',
 '99 Unknown (S/R)',
 '03 T/STAG JUN',
 '06 CROSSROADS',
 '99 UNKNOWN (S/R)',
 '00 NO JUN IN 20M',
 '07 MULTI JUN',
 '05 SLIP ROAD',
 '01 ROUNDABOUT',
 '09 OTHER JUN',
 '02 MINI',
 '08 PRIV DRIVE',
 'Other Jun',
 'Multi Jun',
 'T/Stag Jun',
 'Croassroads',
 'No Jun In 20m',
 'Unknown (S/R)',
 'Priv Drive',
 'Roundabout',
 'Slip Road',
 'Mini']

In [118]:
aliases = pd.read_csv('../data/tfl-aliases.csv')

aliases

,type,consistent_name,alias
0,column,collision_id,AREFNO
1,column,collision_id,Accident Ref.
2,column,collision_id,Accident Ref
3,column,collision_id,_Collision Id
4,column,borough,Borough
...,...,...,...
95,value,other_vehicle,8 OTHER VEHICLE
96,value,other_vehicle,8 Other Vehicle
97,value,other_vehicle,Other Vehicle
98,value,private_hire,9 PRIVATE HIRE


In [ ]:
schemas[schemas['type'] 

In [121]:
d = {}
for type_, alias, consistent_name in aliases.values:
    d[alias] = consistent_name
    
d

{'collision_id': '_Collision Id',
 'borough': 'Borough Name',
 'easting': 'Easting',
 'northing': 'Northing',
 'location': 'Collision Location',
 'collision_severity': '_Collision Severity',
 'junction_detail': 'Junction Detail',
 'date': '_Collision Date',
 'time': 'Time',
 'casulty_id': '_Casualty Id',
 'casualty_class': '_Casualty Class',
 'number_of_casualties': '_Casualty Count',
 'casualty_severity': '_Casualty Severity',
 'mode_of_travel': 'Mode of Travel',
 'fatal': 'Fatal',
 'serious': 'Serious',
 'slight': 'Slight',
 'no_junction_in_20m': 'No Jun In 20m',
 'roundabout': 'Roundabout',
 'mini_roundabout': 'Mini',
 't_or_staggered_junction': 'T/Stag Jun',
 'slip_road': 'Slip Road',
 'crossroads': 'Croassroads',
 'multi_junction': 'Multi Jun',
 'private_drive': 'Priv Drive',
 'other_junction': 'Other Jun',
 'unknown': 'Unknown (S/R)',
 'driver_or_rider': 'Driver/Rider',
 'passenger': 'Passenger',
 'pedestrian': 'Pedestrian',
 'pedal_cycle': 'Pedal Cycle',
 'powered_2_wheeler': 'P

In [111]:
casualties = process_yearly_data(
    path='../data/raw-data/',
    data_type='casualties',
    schema=casualty_schema
)

# casualties['casualty_severity'] = casualties['casualty_severity'].apply(
#     lambda x: format_category(x, severities)
# )

# casualties['casualty_class'] = casualties['casualty_class'].apply(
#     lambda x: format_category(x, casualty_classes)
# )

# casualties['mode_of_travel'] = casualties['mode_of_travel'].apply(
#     lambda x: format_category(x, modes_of_travel)
# )

casualties.sample(5)

,collision_id,casualty_id,casualty_class,number_of_casualties,casualty_severity,mode_of_travel
10783,1210314356,4,Passenger,1,Slight,Car
15986,1180143015,1,1 Driver/Rider,1,3 Slight,4 Car
25206,0114XD80703,1,1 Driver/Rider,1,3 Slight,2 Pedal Cycle
9736,1200277382,1,1 DRIVER/RIDER,1,2 SERIOUS,4 CAR
30940,1170056204,1,1 Driver/Rider,1,3 Slight,4 Car


In [113]:
casualties['mode_of_travel'].unique()

array(['3 POWERED 2 WHEELER', '2 PEDAL CYCLE', '4 CAR', '1 PEDESTRIAN',
       '8 OTHER VEHICLE', '5 TAXI', '6 BUS OR COACH', '7 GOODS VEHICLE',
       '9 PRIVATE HIRE', 'Car', 'Pedestrian', 'Bus Or Coach',
       'Powered 2 Wheeler', 'Goods Vehicle', 'Pedal Cycle',
       'Other Vehicle', 'Private Hire', 'Taxi', '1 Pedestrian', '4 Car',
       '2 Pedal Cycle', '6 Bus Or Coach', '3 Powered 2 Wheeler',
       '7 Goods Vehicle', '5 Taxi', '8 Other Vehicle'], dtype=object)

In [126]:
df = pd.read_csv('../data/tfl-collisions.csv')

df.groupby('collision_id')['year'].count().reset_index().sort_values(by='year')

/var/folders/7z/y_90byrs6yd5xw4yyc9bwl640000gn/T/ipykernel_2751/369968488.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/tfl-collisions.csv')


,collision_id,year
0,1190128300,1
180022,0115SX20800,1
180023,0115SX20801,1
180024,0115SX20802,1
180025,0115SX20803,1
...,...,...
90021,0112BS70048,1
90022,0112BS70050,1
90023,0112BS70051,1
90025,0112BS70053,1


In [128]:
from convertbng.util import convert_lonlat

In [130]:
convert_lonlat([521508], [193079])

([-0.246102], [51.62342462])

In [134]:
df['longitude'], df['latitude'] = convert_lonlat(df['easting'], df['northing'])

df

,collision_id,borough,easting,northing,location,collision_severity,junction_detail,date,time,year,longitude,latitude
0,1210287148,BARNET,521508,193079,"ON TRETAWN GARDENS, 15 METRES SOUTH OF THE JUN...",slight,other_junction,2021-01-01,02:05:00,2021,-0.246102,51.623425
1,1210287149,TOWER HAMLETS,535379,180783,"ON THE HIGHWAY, NEAR THE JUNCTION WITH GLAMIS ...",serious,multi_junction,2021-01-01,03:30:00,2021,-0.050574,51.509767
2,1210287151,LAMBETH,529701,170398,"ON STREATHAM VALE, 5 METRES WEST OF THE JUNCTI...",serious,other_junction,2021-01-01,04:07:00,2021,-0.136152,51.417769
3,1210287155,KENSINGTON & CHELSEA,525312,178385,"ON WARWICK ROAD, NEAR THE JUNCTION WITH TREBOV...",fatal,other_junction,2021-01-01,04:26:00,2021,-0.196411,51.490536
4,1210287157,HOUNSLOW,512144,171526,"ON SWAN ROAD, NEAR THE JUNCTION WITH GROVE CRE...",slight,t_or_staggered_junction,2021-01-01,03:10:00,2021,-0.388169,51.431649
...,...,...,...,...,...,...,...,...,...,...,...,...
270041,0111YE80907,ENFIELD,535140,191850,MERIDIAN WAY J.W GLOVER DRIVE,slight,t_or_staggered_junction,2011-12-30,16:10:00,2011,-0.049767,51.609274
270042,0111YE89047,ENFIELD,533680,194600,GREAT CAMBRIDGE ROAD J/W BURY STREET,slight,crossroads,2011-12-26,03:50:00,2011,-0.069790,51.634335
270043,0111YE89101,ENFIELD,532550,192460,GREAT CAMBRIDGE ROAD 40M S J/W OSTLIFFE ROAD,slight,no_junction_in_20m,2011-11-10,07:29:00,2011,-0.086917,51.615371
270044,0111YR90243,ENFIELD,531570,193500,GREEN LANES J/W GREENWOOD GARDENS,slight,t_or_staggered_junction,2011-05-10,16:35:00,2011,-0.100673,51.624947


In [127]:
df

,collision_id,borough,easting,northing,location,collision_severity,junction_detail,date,time,year
0,1210287148,BARNET,521508,193079,"ON TRETAWN GARDENS, 15 METRES SOUTH OF THE JUN...",slight,other_junction,2021-01-01,02:05:00,2021
1,1210287149,TOWER HAMLETS,535379,180783,"ON THE HIGHWAY, NEAR THE JUNCTION WITH GLAMIS ...",serious,multi_junction,2021-01-01,03:30:00,2021
2,1210287151,LAMBETH,529701,170398,"ON STREATHAM VALE, 5 METRES WEST OF THE JUNCTI...",serious,other_junction,2021-01-01,04:07:00,2021
3,1210287155,KENSINGTON & CHELSEA,525312,178385,"ON WARWICK ROAD, NEAR THE JUNCTION WITH TREBOV...",fatal,other_junction,2021-01-01,04:26:00,2021
4,1210287157,HOUNSLOW,512144,171526,"ON SWAN ROAD, NEAR THE JUNCTION WITH GROVE CRE...",slight,t_or_staggered_junction,2021-01-01,03:10:00,2021
...,...,...,...,...,...,...,...,...,...,...
270041,0111YE80907,ENFIELD,535140,191850,MERIDIAN WAY J.W GLOVER DRIVE,slight,t_or_staggered_junction,2011-12-30,16:10:00,2011
270042,0111YE89047,ENFIELD,533680,194600,GREAT CAMBRIDGE ROAD J/W BURY STREET,slight,crossroads,2011-12-26,03:50:00,2011
270043,0111YE89101,ENFIELD,532550,192460,GREAT CAMBRIDGE ROAD 40M S J/W OSTLIFFE ROAD,slight,no_junction_in_20m,2011-11-10,07:29:00,2011
270044,0111YR90243,ENFIELD,531570,193500,GREEN LANES J/W GREENWOOD GARDENS,slight,t_or_staggered_junction,2011-05-10,16:35:00,2011


In [135]:
import os

In [140]:
os.getenv('HOME')

'/Users/Dan'